# <span style='color:#ff5f27'> Initialization </span>

### Hopsworks Settings

In [ ]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

### Imports

In [ ]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

# <span style='color:#ff5f27'> Retrieve Historical Data </span>

### Retrieve Metadata

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

location_str = secrets.get_secret("BIKES_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
latitude=location['latitude']
longitude=location['longitude']
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

print(f"City: {city}")
print(f"Contry: {country}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
print(f"Day: {today}")

### Connect to Feature Groups

In [ ]:
bikes_fg = fs.get_feature_group(
    name='bikes_trento',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_trento',
    version=1,
)

# <span style='color:#ff5f27'> Get Today's Data </span>

### Bikes Data

In [ ]:
# No API-key is required for this API

network_id = "e-motion-trento"
url = f"https://api.citybik.es/v2/networks/{network_id}"

try:
    resp = requests.get(url).json()
    print("Request Successful!")
except:
    print("Something went wrong, please check the URL.")
    print("Networn Answer:")
    print(requests.get(url))


Check The following info to be the same as your request

In [ ]:
print(f"Network ID: {resp['network']['id']}")

print(f"City: {resp['network']['location']['city']}")
print(f"Contry: {resp['network']['location']['country']}")
print(f"Latitude: {resp['network']['location']['latitude']}")
print(f"Longitude: {resp['network']['location']['longitude']}")

Data Preview

In [ ]:
stations_df = pd.DataFrame(resp['network']['stations'])

stations_df['city'] = city
stations_df['country'] = country
stations_df['tag'] = 'e-motion-trento'

stations_df['timestamp'] = pd.to_datetime(stations_df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f%zZ')
stations_df = stations_df.rename(columns={'timestamp': 'date', 'free_bikes': 'free', 'empty_slots': 'bikes'})

# Convert int64 to int32 as Hopsworks don't parse
stations_df['bikes'] = stations_df['bikes'].astype('int32')
stations_df['free'] = stations_df['free'].astype('int32')
# Extra needs to be string
stations_df['extra'] = stations_df['extra'].astype('str')

stations_df.head()

In [ ]:
stations_df.info()

### Weather Data

In [ ]:
weather_daily_df = util.get_daily_weather_forecast(city=city, latitude=latitude, longitude=longitude)
weather_daily_df

In [ ]:
weather_daily_df.info()

# <span style='color:#ff5f27'> Upload Data </span>

In [ ]:
bikes_fg.insert(stations_df)
print("Updated bike's feature group.")
weather_fg.insert(weather_daily_df, wait=True)
print("Updated weather's feature group.")